<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/DR_coat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time

In [2]:
import pandas as pd
import numpy as np 

from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [3]:
def loss_emputation(w,y,r,e,p):
  return  ((w*r)**2 - 2*w*y*r + y**2 - e)**2/p

def gradient_descent(w, y, le, observed_e, predicted_r, propensities, max_iter, lr=0.1):
  w.requires_grad = True
  y.requires_grad = True
  m = observed_e.shape[0]
  theta = [w.item(), y.item()]
  iter_list = []
  loss_list = []
  for i in range(max_iter):
    loss = 0
    rand1 = np.random.randint(0, m, 1)
    r = torch.from_numpy(predicted_r[rand1]).type(torch.float32)
    e = torch.from_numpy(observed_e[rand1]).type(torch.float32)
    p = torch.from_numpy(propensities[rand1]).type(torch.float32)

    output = le(w,y,r,e,p)
    output.backward()
    w.data -= lr * w.grad
    y.data -= lr * y.grad
    w.grad.fill_(0)
    y.grad.fill_(0)

    for j in range(m):
      every_loss = le(w,y,predicted_r[j],observed_e[j],propensities[j]).item()
      loss = loss + every_loss
    theta[0] = w.item()
    theta[1] = y.item()
    # print("iter_count: ", i, "the loss: ",loss)
    # print("theta:", theta)
    iter_list.append(i)
    loss_list.append(loss)
      
  plt.plot(iter_list, loss_list)
  plt.xlabel("iter")
  plt.ylabel("loss")
  plt.show()
  return theta, loss

In [4]:
# FM 特征组合层
class crossLayer(layers.Layer):
    def __init__(self,input_dim, output_dim=10, **kwargs):
        super(crossLayer, self).__init__(**kwargs)

        self.input_dim = input_dim
        self.output_dim = output_dim
        # 定义交叉特征的权重
        self.kernel = self.add_weight(name='kernel', shape=(self.input_dim, self.output_dim), initializer='glorot_uniform',trainable=True)
        
    def call(self, x): # 对照上述公式中的二次项优化公式一起理解
        a = K.pow(K.dot(x, self.kernel), 2)
        b = K.dot(K.pow(x, 2), K.pow(self.kernel, 2))
        return 0.5 * K.mean(a-b, 1, keepdims=True)

# 定义FM模型
def FM(feature_dim, e, p, O_mask):
    inputs = Input(shape=(feature_dim, ))
    
    # 一阶特征
    linear = Dense(units=1, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))(inputs)
    
    # 二阶特征
    cross = crossLayer(feature_dim)(inputs)
    add = Add()([linear, cross])  # 将一阶特征与二阶特征相加构建FM模型
    
    pred = add
    model = Model(inputs=inputs, outputs=pred)
    
    model.summary()
    # def my_loss_fn(y_true, y_pred):
    #   squared_difference = tf.square(y_true - y_pred)
    #   return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`
    # model.compile(loss=my_loss_fn, optimizer=optimizers.SGD())

    # 以下两段loss定义格式不应更改，keras的loss有且仅有两个参数y_true, y_pred
    def my_coef(y_true, y_pred, predicted_e, propensities, mask):
      error = tf.square(y_true - y_pred)
      error = tf.reshape(error, [290, 300])
      delta = tf.subtract(error, predicted_e)
      a = tf.multiply(mask, delta)
      b = tf.divide(a, propensities)
      return tf.add(predicted_e, b)

    def my_loss_fn(predicted_e, propensities, mask):
      def loss(y_true, y_pred):
        return my_coef(y_true, y_pred, predicted_e, propensities, mask)
      return loss
      
    model.compile(loss=my_loss_fn(predicted_e = e, propensities = p, mask = O_mask),
                  optimizer=optimizers.SGD(), metrics=['accuracy'])

    
    return model    

In [5]:
def result(history):
  # list all data in history
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [6]:
def get_data():
  observed_r = np.genfromtxt("data/train.ascii", dtype=None) # observed ratings
  observed_r = observed_r.astype('float')
  observed_r[observed_r == 0] = np.nan
    
  predicted_r = np.random.randint(1,5,size=observed_r.shape) # predicted rating
    
  propensities = np.genfromtxt("data/our_propensities.ascii", dtype=None) # propensities for all user item pairs

  O_mask = np.int64(observed_r != np.nan)

  item_features = np.genfromtxt("data/item_features_encoded.ascii", dtype=None)
  user_features = np.genfromtxt("data/user_features.ascii", dtype=None)

  test_r = np.genfromtxt("data/test.ascii", dtype=None) # observed ratings for test
  test_r = observed_r.astype('float')
   
  return observed_r, predicted_r, propensities, O_mask, item_features, user_features

In [7]:
def get_features_matrix(O_mask, user_features, item_features):
  features_matrix = []
  for u in range(O_mask.shape[0]):
    for i in range(O_mask.shape[1]):
      user_f = user_features[u]
      item_f = item_features[i]
      observed = np.concatenate((user_f, item_f))
      features_matrix.append(observed)
  features_matrix = np.array(features_matrix)
  return features_matrix

In [8]:
observed_r, predicted_r, propensities, O_mask, item_features, user_features = get_data()
features_matrix = get_features_matrix(O_mask, user_features, item_features)

observed_r_1d = observed_r.flatten()
observed_r_1d = observed_r_1d[~np.isnan(observed_r_1d)]
propensities_1d = np.array(propensities*O_mask).flatten()
propensities_1d = propensities_1d[propensities_1d != 0]
predicted_r_1d = np.array(predicted_r*O_mask).flatten()
predicted_r_1d = predicted_r_1d[predicted_r_1d != 0]

ratings_train = np.genfromtxt("data/train.ascii", dtype=None).astype(np.float32)
ratings_test = np.genfromtxt("data/test.ascii", dtype=None).astype(np.float32)
test_accuracy = 0.5

iter_count = 0
max_iter_joint = 10
while test_accuracy <= 0.95 and iter_count < max_iter_joint:
  observed_e = predicted_r - observed_r # e_ui
  observed_e_1d = observed_e.flatten()
  observed_e_1d = observed_e_1d[~np.isnan(observed_e_1d)]

  # Imputation error
  w = torch.randn(1)
  y = torch.randn(1)
  lr = 0.0001
  max_iter = 1000

  start = time.time()
  theta,loss_list = gradient_descent(w, y, loss_emputation, observed_e_1d, predicted_r_1d, propensities_1d, max_iter, lr)
  end = time.time()
  print("The time of execution of above program is :", end-start)
  print("theta for imputation error: ", theta)

  predicted_e = theta[0]*((predicted_r - theta[1])**2)

  # Training FM
  x_trn = features_matrix
  y_trn = ratings_train.flatten()
  y_tst = ratings_test.flatten()

  # 定义模型
  model = FM(x_trn.shape[1], predicted_e.astype(np.float32), propensities, O_mask.astype(np.float32))

  # 训练模型
  history = model.fit(x_trn, y_trn, epochs=500, batch_size=87000, validation_data=(x_trn, y_tst))
  predicted_r_1d = model.predict(features_matrix)
  predicted_r = predicted_r_1d.reshape(290,300)
  iter_count += 1
  test_accuracy = history.history['val_accuracy'][-1]

  result(history)

NameError: ignored

In [ ]:
counts, bins = np.histogram(predicted_r)
plt.hist(bins[:-1], bins, weights=counts)